In [138]:
import torch
from gensim.models import Word2Vec
import os
import numpy as np
import glob
import pickle
import json
import tqdm
from time import time
import pandas as pd
import torch
import matplotlib.pyplot as plt
from itertools import groupby
from sklearn.cluster import KMeans

In [245]:
pathDF = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/data/word_df_fixed_nonwords.csv" # not really necessary?
pathLabels = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/data/converted_aligned_words_labels_fixed_nonwords.txt"
pathWordLabelsDict = '/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/data/word_labels.pickle'

pathEncodingsDir = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/data/encodings/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/encoded_words"

pathKMeans_10000 = '/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/kmeans_K_10000'
pathKMeans_10500 = '/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/kmeans_K_10500'
pathKMeans_11000 = '/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/kmeans_K_11000'

path_w2v_10000 = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/word2vec_window_5_nepoch_30_vectorsize_100_K10000"
path_w2v_10500 = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/word2vec_window_5_nepoch_30_vectorsize_100_K10500"
path_w2v_11000 = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/word2vec_window_5_nepoch_30_vectorsize_100_K11000"

# weighted Kmeans models
pathKMeans_10000_weighted = '/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/kmeans_K_10000_weighted_True'
pathKMeans_10500_weighted = '/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/kmeans_K_10500_weighted_True'
pathKMeans_11000_weighted = '/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/kmeans_K_11000_weighted_True'

path_w2v_10000_weighted = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/word2vec_window_5_nepoch_30_vectorsize_100_K10000_weighted_True"
path_w2v_10500_weighted = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/word2vec_window_5_nepoch_30_vectorsize_100_K10500_weighted_True"
path_w2v_11000_weighted = "/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/hacpc32neg-6-12-2-4-gt-cosine-norelu-encodeseg-ls/word2vec_window_5_nepoch_30_vectorsize_100_K11000_weighted_True"

## Get the data

In [140]:
encExtension = ".pt"
encFiles = glob.glob(os.path.join(pathEncodingsDir, '*' + encExtension))
encodings = []
if encExtension == ".txt":
    for encfile in tqdm.tqdm(encFiles):
        with open(encfile, 'r') as f:
            lines = f.readlines()
        for line in lines:
            data = line.split()
            encodings.append([float(x) for x in data])
    wordEncs = torch.tensor(encodings)
elif encExtension == ".pt":
    for encfile in tqdm.tqdm(encFiles):
            encodings.append(torch.tensor(torch.load(encfile)))
    wordEncs = torch.cat(encodings)

100%|██████████| 255/255 [00:08<00:00, 30.46it/s]


In [141]:
wordEncs.shape[0] == len(uniqueConsecutiveWords)

True

In [142]:
with open(pathLabels, 'r') as f:
    lines = f.readlines()
output = {}  # Step in librispeech dataset is 160bits
maxPhone = 0
allLabels = []
for line in lines:
    data = line.split()
    recordingLabels = [int(x) for x in data[1:]]
    output[data[0]] = recordingLabels
    allLabels += recordingLabels
    maxPhone = max(maxPhone, max(output[data[0]]))

In [143]:
file = open(pathWordLabelsDict,'rb')
word2Labels = pickle.load(file)
file.close()
labels2Words = {v: k for k, v in word2Labels.items()}

In [205]:
uniqueConsecutiveLabels = [x[0] for x in groupby(allLabels)]
len(uniqueConsecutiveLabels)

348122

In [145]:
df = pd.read_csv(pathDF)
df.shape

(350924, 3)

In [146]:
df.shape[0] - len(uniqueConsecutiveLabels)

2802

In [206]:
uniqueConsecutiveWords = list(map(labels2Words.get, uniqueConsecutiveLabels))

## Split long list with everything into digestable shorter sentences for word2vec

In [149]:
n_words_in_sentence = 15
splitWords = []
lastIdx = 0
for i in range(len(uniqueConsecutiveWords) // n_words_in_sentence):
    sentence = uniqueConsecutiveWords[i * n_words_in_sentence : (i+1) * n_words_in_sentence]
    splitWords.append(sentence)
    lastIdx = (i+1) * n_words_in_sentence
splitWords.append(uniqueConsecutiveWords[lastIdx:])

## word2vec model trained on regular text

In [150]:
text_model = Word2Vec(min_count=1, window=5, vector_size=100)
text_model.build_vocab(corpus_iterable=splitWords)
text_model.train(corpus_iterable=splitWords, total_examples=text_model.corpus_count, total_words=text_model.corpus_total_words, epochs=30)

text_model.save("/pio/scratch/1/i325922/data/BUCKEYE/raw/word2vec_encodings/models/text_model/w2v.model")

In [156]:
investigatedWord = 'small'
text_model.wv.most_similar(investigatedWord, topn=5)

[('large', 0.7316713333129883),
 ('such', 0.7254460453987122),
 ('rural', 0.6970985531806946),
 ('christian', 0.6497025489807129),
 ("england's", 0.6425490379333496)]

In [254]:
def get_kmeans_results(pathCheckpoint, encodings, wordLabels, preds=True):
    with open(pathCheckpoint + '/kmeans.pickle', 'rb') as handle:
        model = pickle.load(handle)
    if preds:
        labels = model.predict(encodings)
    else:
        labels = model.labels_
    cluster2WordDict = {}
    npLabels = np.array(wordLabels)
    for i in range(10000):
        wordsWhereCluster = (np.where(labels == i)[0])
        cluster2WordDict[i] = list(map(labels2Words.get, npLabels[wordsWhereCluster]))
    return model, labels, cluster2WordDict

# K = 10000

In [258]:
k10000, k10000_preds, k10000_cluster2WordDict = get_kmeans_results(pathKMeans_10000, wordEncs, uniqueConsecutiveLabels, preds=False)

In [266]:
k10000_weighted, k10000_weighted_preds, k10000_weighted_cluster2WordDict = get_kmeans_results(pathKMeans_10000_weighted, wordEncs, uniqueConsecutiveLabels, preds=False)

In [267]:
k10000_weighted_cluster2WordDict[666]

['<VOCNOISE>',
 '<SIL>',
 '<SIL>',
 '<SIL>',
 'she',
 'comes',
 'yknow',
 'types',
 '<VOCNOISE>',
 '<IVER>',
 'can',
 'right',
 '<SIL>',
 'it',
 '<SIL>',
 '<VOCNOISE>',
 'got',
 '<SIL>',
 'they',
 'could',
 'i',
 'a',
 'order',
 'ask',
 'independent',
 'past',
 'loves',
 'that',
 'ships',
 '<VOCNOISE>',
 'child',
 '<IVER>',
 'but',
 'was',
 "she's",
 'and',
 'it',
 'they',
 'grade',
 'they',
 'it',
 'state',
 'when',
 'more']

In [260]:
k10000_cluster2WordDict[666]

['school',
 "won't",
 'yknow',
 'you',
 '<VOCNOISE>',
 'want',
 'make',
 'the',
 'about',
 'yknow',
 'when',
 'not',
 'flips',
 'gonna',
 '<SIL>',
 'game',
 'no',
 '<SIL>',
 'see',
 '<SIL>',
 '<SIL>',
 'and',
 'you',
 '<IVER>',
 'i',
 'the',
 'mentality',
 'just',
 'the',
 'um-hum',
 'lot',
 'while',
 'have',
 'the',
 'of',
 'knew',
 '<VOCNOISE>',
 '<VOCNOISE>',
 'um',
 'least',
 '<SIL>',
 'yknow']

Word2vecs were trained with 'words' being hexadecimal notation of Kmeans cluster IDs

In [298]:
def decode_encodings_w2v(clusterId, model, c2vDict, topn=5):
    simHexes = model.wv.most_similar(hex(clusterId), topn=topn)
    decodedHexes = [(int(s, 16), c2vDict[int(s, 16)]) for (s, _) in simHexes]
    print(f"Queried Cluster ID: {clusterId}\n")
    print(f"Corresponding unique word types assigned to it: {[x[0] for x in groupby(c2vDict[clusterId])]}\n")
    print(f"Top {topn} matching clusters (with corresponding unique word types assigned to it):")
    for (c, ws) in decodedHexes:
        print(c)
        print([x[0] for x in groupby(ws)])

In [290]:
w2v_10000 = Word2Vec.load(str(path_w2v_10000 + '/w2v.model'))
w2v_10000_weighted = Word2Vec.load(str(path_w2v_10000_weighted + '/w2v.model'))

In [299]:
Id = 88
decode_encodings_w2v(Id, w2v_10000, k10000_cluster2WordDict)

Queried Cluster ID: 88

Corresponding unique word types assigned to it: ['<SIL>', 'all', 'and', 'um-hum', 'always', 'was', 'look', 'along', 'everybody', '<VOCNOISE>', 'hold', '<SIL>', '<VOCNOISE>', 'brother', 'the', 'these', 'when', 'a', "i've", 'but', 'being', 'um']

Top 5 matching clusters (with corresponding unique word types assigned to it):
8387
['<VOCNOISE>', '<SIL>', '<VOCNOISE>', '<IVER>', '<SIL>', 'of', '<SIL>', 'stuff', '<VOCNOISE>', '<SIL>', "isn't", '<SIL>', 'everything', 'world', 'because', '<SIL>', 'expanding', 'the', 'um-hum', 'a', 'after', 'yeah', 'was', 'rudest', 'my', '<SIL>', "isn't", 'these', 'great', 'schools', 'so', 'think', 'but', 'hold', 'her', 'to', 'in', 'ground', 'people']
1349
['either', 'trimester', 'my', 'planes', 'time', 'sometimes', 'downstairs', 'times', 'the', 'brand', 'and', 'transmeridian', 'apartment', 'i', 'times', 'um-hum', 'higher', 'program', 'have', '<VOCNOISE>', 'side', '<SIL>', 'store', '<IVER>', 'about', 'over', "don't", '<LAUGH>', 'thing', 

In [300]:
decode_encodings_w2v(Id, w2v_10000_weighted, k10000_weighted_cluster2WordDict)

Queried Cluster ID: 88

Corresponding unique word types assigned to it: ['<IVER>', 'own', '<VOCNOISE>', 'me', "you're", 'then', 'so', 'kinda', 'that', 'are', 'time', 'things', 'principles', '<SIL>', 'back', 'what', 'because', 'there', 'wanna', 'and', 'to', 'on', 'social', '<SIL>', 'be', '<VOCNOISE>', 'my', 'you', 'she', 'for', 'man', 'and']

Top 5 matching clusters (with corresponding unique word types assigned to it):
873
['they', '<VOCNOISE>', 'serious', 'really', 'it', 'whole', 'for', 'and', 'up', 'thirty', 'you', '<VOCNOISE>', 'talk', 'resources', 'of', 'think', 'need', 'can', 'just', 'me', '<SIL>', 'and', 'dad', 'time', 'voted', 'election', 'sit', 'we', "there's", '<SIL>', 'should', 'like', '<IVER>', 'in', 'faith', 'know', '<SIL>', '<IVER>', 'what', 'cool', 'integrated', 'that', 'nobody', '<IVER>', 'here', 'kind', 'i', 'um-hmm', 'where', '<IVER>', "didn't", 'but', 'bad', '<SIL>', 'and', 'think', '<IVER>', 'apparently', 'in', 'it', 'and', 'the', 'just', '<VOCNOISE>', 'every', 'my',

## K = 11000

In [293]:
k11000, k11000_preds, k11000_cluster2WordDict = get_kmeans_results(pathKMeans_11000, wordEncs, uniqueConsecutiveLabels, preds=False)

In [294]:
k11000_weighted, k11000_weighted_preds, k11000_weighted_cluster2WordDict = get_kmeans_results(pathKMeans_11000_weighted, wordEncs, uniqueConsecutiveLabels, preds=False)

In [295]:
w2v_11000 = Word2Vec.load(str(path_w2v_11000 + '/w2v.model'))
w2v_11000_weighted = Word2Vec.load(str(path_w2v_11000_weighted + '/w2v.model'))

In [297]:
Id = 97
decode_encodings_w2v(Id, w2v_11000, k11000_weighted_cluster2WordDict)

Queried Custer ID: 97

Corresponding word types assigned to it: ['<SIL>', '<VOCNOISE>', '<IVER>', '<VOCNOISE>', '<SIL>', '<VOCNOISE>', '<IVER>', '<SIL>', 'and', '<SIL>', '<IVER>', '<SIL>', 'there', 'a']

Top 5 matching clusters (with corresponding unique word types assigned to it):
7429
["grandfather's", 'other', 'any', 'right', 'do', 'uh', 'believe', 'age', 'anywhere', 'would', 'mind', 'not', 'england', 'mean', 'has', 'that', 'uh', '<VOCNOISE>', 'how', 'balsamic', 'kind', '<SIL>', 'i', 'climbing', 'yeah', 'out', 'grew', 'when', '<VOCNOISE>', 'positions', 'is', 'here', 'our', 'was', 'apartment', 'delivered', 'i', 'and', 'she', 'i', 'a', 'child', 'now', 'but', 'and', 'right', '<NOISE>', '<SIL>', "didn't", 'seventies', 'uh', 'even', 'an', 'marriage', '<VOCNOISE>', 'yeah', 'and', 'connecticut', 'its', 'way', 'she', 'as', 'just', 'not', 'me', 'too', 'easy', 'just', 'yknow', 'their', '<VOCNOISE>', 'she', 'a', 'was', 'of', 'yeah', "i'm", 'i', 'of', 'and', 'they', '<SIL>', '<VOCNOISE>', 'okay